# Intel Görüntü Sınıflandırması - CNN Derin Öğrenme Projesi

## Akbank Derin Öğrenme Bootcamp Projesi

Bu proje kapsamında, CNN (Convolutional Neural Network) mimarisi kullanarak Intel görüntü veri seti üzerinde 6 farklı doğa kategorisini sınıflandıran bir derin öğrenme modeli geliştirilecektir.

### Veri Seti Bilgileri:
- **Kaynak:** Intel Image Classification Dataset (Kaggle)
- **Kategoriler:** Buildings, Forest, Glacier, Mountain, Sea, Street (6 sınıf)
- **Boyut:** 25.000 görüntü (150x150 piksel)
- **Dağılım:** ~14k train, ~3k test, ~7k prediction

### Proje Aşamaları:
1. Veri Önişleme ve Görselleştirme
2. Data Augmentation
3. Model Tasarımı ve Eğitimi
4. Model Değerlendirmesi
5. Grad-CAM Görselleştirme
6. Prediction Verisi Üzerinde Test

## 1. Kütüphanelerin İçe Aktarılması

Projenin tüm aşamalarında kullanacağımız kütüphaneleri içe aktaralım.

In [ ]:
# Temel veri işleme ve görselleştirme kütüphaneleri
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import cv2
import os
import warnings
warnings.filterwarnings('ignore')

# TensorFlow ve Keras kütüphaneleri
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Dense, Conv2D, MaxPooling2D, Flatten, Dropout, BatchNormalization
from tensorflow.keras.layers import GlobalAveragePooling2D
from tensorflow.keras.optimizers import Adam, SGD
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import VGG16  
from tensorflow.keras.regularizers import l2
from tensorflow.keras.utils import to_categorical

# Sklearn kütüphaneleri
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.utils.class_weight import compute_class_weight

# Grad-CAM için gerekli kütüphaneler
from tensorflow.keras.models import load_model
import tensorflow.keras.backend as K

# Rastgelelik kontrolü için seed değerleri
np.random.seed(42)
tf.random.set_seed(42)

print("Kütüphaneler başarıyla yüklendi!")
print(f"TensorFlow versiyon: {tf.__version__}")
print(f"GPU mevcut: {tf.config.list_physical_devices('GPU')}")

## 2. Veri Setinin Yüklenmesi ve Keşfi

Intel görüntü veri setinin yapısını inceleyelim ve temel bilgileri elde edelim.

In [ ]:
# Veri seti yollarını tanımlayalım
base_path = '/kaggle/input/intel-image-classification'
train_path = '/kaggle/input/intel-image-classification/seg_train/seg_train'
test_path = '/kaggle/input/intel-image-classification/seg_test/seg_test'
pred_path = '/kaggle/input/intel-image-classification/seg_pred/seg_pred'

# Sınıf isimleri ve kodları
class_names = ['buildings', 'forest', 'glacier', 'mountain', 'sea', 'street']
class_dict = {
    'buildings': 0,
    'forest': 1,
    'glacier': 2,
    'mountain': 3,
    'sea': 4,
    'street': 5
}

print("Veri Seti Yolları:")
print(f"Ana yol: {base_path}")
print(f"Eğitim verisi: {train_path}")
print(f"Test verisi: {test_path}")
print(f"Tahmin verisi: {pred_path}")
print(f"\nSınıflar: {class_names}")

# Veri seti yapısını inceleyelim
def explore_dataset_structure(path, dataset_name):
    """Veri seti yapısını ve her sınıftaki görüntü sayısını inceleyen fonksiyon"""
    print(f"\n{dataset_name} Veri Seti Analizi:")
    print("-" * 40)
    
    try:
        folders = os.listdir(path)
        total_images = 0
        
        for folder in sorted(folders):
            folder_path = os.path.join(path, folder)
            if os.path.isdir(folder_path):
                image_count = len(os.listdir(folder_path))
                total_images += image_count
                print(f"{folder}: {image_count} görüntü")
        
        print(f"Toplam: {total_images} görüntü")
        return total_images
        
    except Exception as e:
        print(f"Hata: {e}")
        return 0

# Her veri setini inceleyelim
train_count = explore_dataset_structure(train_path, "EĞİTİM")
test_count = explore_dataset_structure(test_path, "TEST")

# Prediction klasörü farklı yapıda olduğu için ayrı inceleyeceğiz
print(f"\nTAHMİN Veri Seti Analizi:")
print("-" * 40)
try:
    pred_images = os.listdir(pred_path)
    pred_count = len([f for f in pred_images if f.lower().endswith(('.jpg', '.jpeg', '.png'))])
    print(f"Toplam tahmin görüntüsü: {pred_count}")
except Exception as e:
    print(f"Hata: {e}")
    pred_count = 0

print(f"\nGENEL ÖZET:")
print(f"Toplam eğitim görüntüsü: {train_count}")
print(f"Toplam test görüntüsü: {test_count}")
print(f"Toplam tahmin görüntüsü: {pred_count}")
print(f"Genel toplam: {train_count + test_count + pred_count}")

## 3. Veri Görselleştirme ve İstatistiksel Analiz

Her kategoriden örnek görüntüleri görselleştirelim ve veri setinin dağılımını analiz edelim.

In [ ]:
# Her sınıftan örnek görüntüleri görselleştiren fonksiyon
def visualize_sample_images(path, class_names, num_samples=3):
    """Her sınıftan örnek görüntüleri gösteren fonksiyon"""
    fig, axes = plt.subplots(len(class_names), num_samples, figsize=(15, 20))
    fig.suptitle('Her Sınıftan Örnek Görüntüler', fontsize=16, fontweight='bold')
    
    for i, class_name in enumerate(class_names):
        class_path = os.path.join(path, class_name)
        
        try:
            # Sınıf klasöründeki görüntüleri listele
            images = os.listdir(class_path)
            images = [img for img in images if img.lower().endswith(('.jpg', '.jpeg', '.png'))]
            
            # Rastgele örnek seç
            sample_images = np.random.choice(images, min(num_samples, len(images)), replace=False)
            
            for j, img_name in enumerate(sample_images):
                img_path = os.path.join(class_path, img_name)
                
                # Görüntüyü yükle ve göster
                img = cv2.imread(img_path)
                img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
                
                axes[i, j].imshow(img)
                axes[i, j].set_title(f'{class_name.title()}', fontweight='bold')
                axes[i, j].axis('off')
                
        except Exception as e:
            print(f"Hata ({class_name}): {e}")
            
    plt.tight_layout()
    plt.show()

# Örnek görüntüleri göster
visualize_sample_images(train_path, class_names, num_samples=4)

In [ ]:
# Sınıf dağılımlarını görselleştiren fonksiyon
def plot_class_distribution(path, title, class_names):
    """Sınıf dağılımını görselleştiren fonksiyon"""
    class_counts = []
    
    for class_name in class_names:
        class_path = os.path.join(path, class_name)
        try:
            count = len(os.listdir(class_path))
            class_counts.append(count)
        except:
            class_counts.append(0)
    
    # Bar grafiği
    plt.figure(figsize=(12, 6))
    
    plt.subplot(1, 2, 1)
    bars = plt.bar(class_names, class_counts, color=plt.cm.viridis(np.linspace(0, 1, len(class_names))))
    plt.title(f'{title} - Sınıf Dağılımı', fontweight='bold')
    plt.xlabel('Sınıflar')
    plt.ylabel('Görüntü Sayısı')
    plt.xticks(rotation=45)
    
    # Her bar'ın üzerine değer yazdır
    for bar, count in zip(bars, class_counts):
        plt.text(bar.get_x() + bar.get_width()/2., bar.get_height() + 50,
                f'{count}', ha='center', va='bottom', fontweight='bold')
    
    # Pasta grafiği
    plt.subplot(1, 2, 2)
    colors = plt.cm.viridis(np.linspace(0, 1, len(class_names)))
    plt.pie(class_counts, labels=class_names, autopct='%1.1f%%', colors=colors, startangle=90)
    plt.title(f'{title} - Yüzdelik Dağılım', fontweight='bold')
    
    plt.tight_layout()
    plt.show()
    
    return class_counts

# Train ve test setlerinin dağılımını görselleştir
print("EĞİTİM VERİSİ DAĞILIMI:")
train_counts = plot_class_distribution(train_path, "Eğitim Verisi", class_names)

print("\nTEST VERİSİ DAĞILIMI:")
test_counts = plot_class_distribution(test_path, "Test Verisi", class_names)

# İstatistiksel bilgileri göster
print("\nİSTATİSTİKSEL ÖZET:")
print("-" * 50)
for i, class_name in enumerate(class_names):
    train_count = train_counts[i]  
    test_count = test_counts[i]
    total = train_count + test_count
    train_ratio = (train_count / total) * 100 if total > 0 else 0
    test_ratio = (test_count / total) * 100 if total > 0 else 0
    
    print(f"{class_name.upper()}:")
    print(f"  Eğitim: {train_count} (%{train_ratio:.1f})")
    print(f"  Test: {test_count} (%{test_ratio:.1f})")
    print(f"  Toplam: {total}")
    print()

## 4. Veri Önişleme ve Etiketleme

Görüntüleri model için uygun formata dönüştürelim ve etiketleri sayısal değerlere çevirelim.

In [ ]:
# Görüntü boyutları ve parametreler
IMG_HEIGHT = 150
IMG_WIDTH = 150
IMG_CHANNELS = 3


def load_and_preprocess_data(data_path, class_names, limit_per_class=None):
    """
    Veri setini yükleyen ve önişleyen fonksiyon
    
    Args:
        data_path: Veri seti klasör yolu
        class_names: Sınıf isimleri listesi
        limit_per_class: Her sınıftan alınacak maksimum görüntü sayısı (None = tümü)
    
    Returns:
        images: Numpy array olarak görüntüler
        labels: Numpy array olarak etiketler
    """
    images = []
    labels = []
    
    print(f"Veri yükleniyor: {data_path}")
    
    for class_idx, class_name in enumerate(class_names):
        class_path = os.path.join(data_path, class_name)
        print(f"  {class_name} sınıfı yükleniyor...")
        
        try:
            # Sınıf klasöründeki tüm görüntüleri listele
            image_files = [f for f in os.listdir(class_path) 
                          if f.lower().endswith(('.jpg', '.jpeg', '.png'))]
            
            # Eğer limit varsa, rastgele örnekle
            if limit_per_class and len(image_files) > limit_per_class:
                image_files = np.random.choice(image_files, limit_per_class, replace=False)
            
            class_images = []
            class_labels = []
            
            for img_file in image_files:
                img_path = os.path.join(class_path, img_file)
                
                # Görüntüyü yükle
                img = cv2.imread(img_path)
                if img is not None:
                    # BGR'dan RGB'ye dönüştür
                    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
                    
                    # Boyutlandır
                    img = cv2.resize(img, (IMG_WIDTH, IMG_HEIGHT))
                    
                    # Normalize etme - ImageDataGenerator'da yapılacak
                    img = img.astype('float32')  # 0-255 aralığında bırak
                    
                    class_images.append(img)
                    class_labels.append(class_idx)
            
            images.extend(class_images)
            labels.extend(class_labels)
            
            print(f"    {len(class_images)} görüntü yüklendi")
            
        except Exception as e:
            print(f"    Hata: {e}")
    
    print(f"Toplam {len(images)} görüntü yüklendi")
    
    return np.array(images), np.array(labels)

# Eğitim verisini yükle
print("EĞİTİM VERİSİ YÜKLENİYOR...")
X_train_full, y_train_full = load_and_preprocess_data(train_path, class_names)

# Test verisini yükle
print("\nTEST VERİSİ YÜKLENİYOR...")
X_test, y_test = load_and_preprocess_data(test_path, class_names)

print(f"\nVERİ ÖNİŞLEME TAMAMLANDI!")
print(f"Eğitim verisi şekli: {X_train_full.shape}")
print(f"Eğitim etiketleri şekli: {y_train_full.shape}")
print(f"Test verisi şekli: {X_test.shape}")
print(f"Test etiketleri şekli: {y_test.shape}")

# Etiketleri kategorik formata dönüştür
y_train_full_categorical = to_categorical(y_train_full, num_classes=len(class_names))
y_test_categorical = to_categorical(y_test, num_classes=len(class_names))

print(f"Kategorik etiket şekli: {y_train_full_categorical.shape}")

# Bazı istatistikleri göster
print(f"\nVERİ İSTATİSTİKLERİ:")
print(f"Görüntü boyutu: {IMG_HEIGHT}x{IMG_WIDTH}x{IMG_CHANNELS}")
print(f"Piksel değeri aralığı: {X_train_full.min():.3f} - {X_train_full.max():.3f}")
print(f"Sınıf sayısı: {len(class_names)}")
print(f"Sınıf dağılımı (eğitim): {np.bincount(y_train_full)}")
print(f"Sınıf dağılımı (test): {np.bincount(y_test)}")

## 5. Veri Artırma (Data Augmentation)

Model performansını artırmak için veri artırma tekniklerini uygulayalım.

In [ ]:
# Veri artırma (Data Augmentation) ayarları - VGG16 eğitimi ve örnek görselleştirme

# Genel augmentation (0-1 normalize + çeşitli dönüşümler)
augmentation_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=25,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True,
    zoom_range=0.2,
    brightness_range=[0.8, 1.2],
    shear_range=0.1,
    fill_mode='reflect'
)

# Validation için yalnızca normalize (örnek amaçlı)
validation_datagen = ImageDataGenerator(rescale=1./255)

# VGG16 için (girdi 0-255 -> Lambda içinde preprocess_input uygulanacak) ayrı jeneratörler
vgg_train_datagen = ImageDataGenerator(
    rotation_range=25,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True,
    zoom_range=0.2,
    brightness_range=[0.8, 1.2],
    shear_range=0.1,
    fill_mode='reflect'
)
vgg_val_datagen = ImageDataGenerator()


def show_augmentation_examples():
    """Augmentation örneklerini gösterir (girdi: orijinal 0-255 RGB, ekranda normalize edilmiş versiyonlar)."""
    random_idx = np.random.randint(0, len(X_train_full))
    sample_img = X_train_full[random_idx]
    sample_label = y_train_full[random_idx]
    class_name = class_names[sample_label]

    # Orijinal görüntü (0-255 formatı)
    if sample_img.max() <= 1.0:
        display_img = (sample_img * 255).astype(np.uint8)
    else:
        display_img = sample_img.astype(np.uint8)

    img_for_aug = display_img.copy()
    img_array = np.expand_dims(img_for_aug, axis=0)

    fig, axes = plt.subplots(1, 6, figsize=(18, 3))
    axes[0].imshow(display_img)
    axes[0].set_title(f'Orijinal\n{class_name}', fontweight='bold', fontsize=10)
    axes[0].axis('off')

    for i in range(5):
        aug_iter = augmentation_datagen.flow(img_array, batch_size=1, shuffle=False)
        aug_img = next(aug_iter)[0]  # 0-1 aralığında
        aug_img = np.clip(aug_img, 0, 1)
        axes[i+1].imshow(aug_img)
        axes[i+1].set_title(f'Aug {i+1}', fontweight='bold', fontsize=10)
        axes[i+1].axis('off')

    plt.tight_layout()
    plt.show()

print("VERİ ARTIRMA ÖRNEKLERİ (VGG16 için de uygulanabilir genel pipeline):\n")
for example_num in range(3):
    print(f"Örnek {example_num + 1}:")
    show_augmentation_examples()
    print()

## 6. Train-Validation-Test Ayrımı

Eğitim verisini train ve validation setlerine ayıralım.

In [ ]:
# Eğitim verisini train ve validation setlerine ayır (80% train, 20% validation)
X_train, X_val, y_train, y_val = train_test_split(
    X_train_full, y_train_full_categorical, 
    test_size=0.2, 
    random_state=42, 
    stratify=y_train_full  # Her sınıftan eşit oranda örnekleme
)

print("VERİ SETİ AYIRIMI TAMAMLANDI!")
print(f"Eğitim seti: {X_train.shape[0]} görüntü")
print(f"Validation seti: {X_val.shape[0]} görüntü")
print(f"Test seti: {X_test.shape[0]} görüntü")

# Sınıf dağılımlarını kontrol et
print(f"\nSINIF DAĞILIMLARI:")
print("Train seti:")
train_class_counts = np.sum(y_train, axis=0)
for i, class_name in enumerate(class_names):
    print(f"  {class_name}: {int(train_class_counts[i])}")

print("Validation seti:")
val_class_counts = np.sum(y_val, axis=0)
for i, class_name in enumerate(class_names):
    print(f"  {class_name}: {int(val_class_counts[i])}")

print("Test seti:")
test_class_counts = np.sum(y_test_categorical, axis=0)
for i, class_name in enumerate(class_names):
    print(f"  {class_name}: {int(test_class_counts[i])}")

# Sınıf ağırlıklarını hesapla (dengesiz veri seti için)
class_weights = compute_class_weight(
    class_weight='balanced',
    classes=np.unique(y_train_full),
    y=y_train_full
)

class_weight_dict = dict(zip(np.unique(y_train_full), class_weights))
print(f"\nSINIF AĞIRLIKLARI:")
for i, class_name in enumerate(class_names):
    print(f"  {class_name}: {class_weight_dict[i]:.3f}")

# Veri boyutlarını görselleştir
plt.figure(figsize=(12, 5))

# Bar grafiği
plt.subplot(1, 2, 1)
datasets = ['Train', 'Validation', 'Test']
sizes = [X_train.shape[0], X_val.shape[0], X_test.shape[0]]
colors = ['skyblue', 'lightgreen', 'lightcoral']

bars = plt.bar(datasets, sizes, color=colors)
plt.title('Veri Seti Boyutları', fontweight='bold')
plt.ylabel('Görüntü Sayısı')

# Bar'ların üzerine değer yazdır
for bar, size in zip(bars, sizes):
    plt.text(bar.get_x() + bar.get_width()/2., bar.get_height() + 100,
             f'{size}', ha='center', va='bottom', fontweight='bold')

# Pasta grafiği
plt.subplot(1, 2, 2)
plt.pie(sizes, labels=datasets, autopct='%1.1f%%', colors=colors, startangle=90)
plt.title('Veri Seti Dağılımı', fontweight='bold')

plt.tight_layout()
plt.show()

## 7. Transfer Learning Modeli (VGG16)

Pre-trained VGG16 modeli kullanarak transfer learning yaklaşımını uygulayalım.

In [ ]:
# Gerekli temel parametreler
input_shape = (IMG_HEIGHT, IMG_WIDTH, IMG_CHANNELS)
num_classes = len(class_names)
BATCH_SIZE = 64  # İstersen değiştirilebilir


def create_transfer_learning_model_vgg16(input_shape, num_classes):
    # VGG16 tabanını yükle (top katmanlar hariç)
    base_model = VGG16(weights='imagenet', include_top=False, input_shape=input_shape)
    base_model.trainable = False  # İlk aşama: feature extractor
    model = Sequential([
        layers.Input(shape=input_shape),
        layers.Lambda(lambda x: tf.keras.applications.vgg16.preprocess_input(x * 255.0), name='vgg16_preprocess'),
        base_model,
        GlobalAveragePooling2D(),
        Dense(512, activation='relu'),
        BatchNormalization(),
        Dropout(0.5),
        Dense(256, activation='relu'),
        Dropout(0.3),
        Dense(num_classes, activation='softmax')
    ])
    return model

print("TRANSFER LEARNING MODELİ OLUŞTURULUYOR (VGG16)...")
vgg_transfer_model = create_transfer_learning_model_vgg16(input_shape, num_classes)
vgg_transfer_model.compile(optimizer=Adam(learning_rate=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])
print("\nVGG16 TRANSFER MODEL ÖZETİ:")
vgg_transfer_model.summary()

vgg_transfer_callbacks = [
    EarlyStopping(monitor='val_loss', patience=8, restore_best_weights=True, verbose=1),
    ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=4, min_lr=1e-7, verbose=1),
    ModelCheckpoint('vgg16_transfer_best.h5', monitor='val_accuracy', save_best_only=True, verbose=1)
 ]

# VGG16 için generatorlar (X_train şu anda 0-1 aralığında; Lambda içinde *255 yapılıyor)
vgg_train_generator = vgg_train_datagen.flow(
    X_train, y_train,
    batch_size=BATCH_SIZE,
    shuffle=True
)
vgg_val_generator = vgg_val_datagen.flow(
    X_val, y_val,
    batch_size=BATCH_SIZE,
    shuffle=False
)

import math
print("\nVGG16 TRANSFER LEARNING MODELİ EĞİTİLİYOR...")
vgg_transfer_history = vgg_transfer_model.fit(
    vgg_train_generator,
    steps_per_epoch=math.ceil(len(X_train) / BATCH_SIZE),
    epochs=50,
    validation_data=vgg_val_generator,
    validation_steps=math.ceil(len(X_val) / BATCH_SIZE),
    callbacks=vgg_transfer_callbacks,
    class_weight=class_weight_dict,
    verbose=1
)

print("\nVGG16 TRANSFER LEARNING EĞİTİMİ TAMAMLANDI!")
vgg_transfer_final_val_accuracy = max(vgg_transfer_history.history['val_accuracy'])
vgg_transfer_best_epoch = vgg_transfer_history.history['val_accuracy'].index(vgg_transfer_final_val_accuracy) + 1
print(f"VGG16 - En iyi validation accuracy: {vgg_transfer_final_val_accuracy:.4f} (Epoch {vgg_transfer_best_epoch})")

## 8. Model Performans Değerlendirmesi

Eğitim sürecini analiz edelim ve accuracy/loss grafiklerini çizdirelim (VGG16 Transfer Modeli).

In [ ]:
def plot_training_history_simple(history, title="VGG16 Transfer Model Eğitim Süreci"):
    plt.figure(figsize=(14,5))
    plt.subplot(1,2,1)
    plt.plot(history.history['accuracy'], label='Train Accuracy', linewidth=2)
    plt.plot(history.history['val_accuracy'], label='Validation Accuracy', linewidth=2)
    plt.title(f'{title} - Accuracy', fontweight='bold')
    plt.xlabel('Epoch'); plt.ylabel('Accuracy')
    plt.legend(); plt.grid(True, alpha=0.3)
    plt.subplot(1,2,2)
    plt.plot(history.history['loss'], label='Train Loss', linewidth=2)
    plt.plot(history.history['val_loss'], label='Validation Loss', linewidth=2)
    plt.title(f'{title} - Loss', fontweight='bold')
    plt.xlabel('Epoch'); plt.ylabel('Loss')
    plt.legend(); plt.grid(True, alpha=0.3)
    plt.tight_layout(); plt.show()

print("VGG16 TRANSFER EĞİTİM SÜRECİ:")
plot_training_history_simple(vgg_transfer_history, "VGG16 Transfer")

## 9. Confusion Matrix ve Classification Report

Test verisi üzerinde VGG16 transfer modelinin performansını analiz edelim.

In [ ]:
# TEST VERİSİ ÜZERİNDE DEĞERLENDİRME (VGG16 TEK MODEL)
print("TEST VERİSİ ÜZERİNDE TAHMİNLER YAPILIYOR (VGG16)...")

all_results = {}


def plot_confusion_matrix(y_true, y_pred, class_names, title="Confusion Matrix"):
    cm = confusion_matrix(y_true, y_pred)
    cm_normalized = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
    fig, axes = plt.subplots(1, 2, figsize=(15, 6))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=class_names, yticklabels=class_names, ax=axes[0])
    axes[0].set_title(f'{title} (Ham)', fontweight='bold')
    axes[0].set_xlabel('Tahmin'); axes[0].set_ylabel('Gerçek')
    sns.heatmap(cm_normalized, annot=True, fmt='.3f', cmap='Blues', xticklabels=class_names, yticklabels=class_names, ax=axes[1])
    axes[1].set_title(f'{title} (Normalize)', fontweight='bold')
    axes[1].set_xlabel('Tahmin'); axes[1].set_ylabel('Gerçek')
    plt.tight_layout(); plt.show()
    return cm, cm_normalized

# VGG16 tahminleri
preds = vgg_transfer_model.predict(X_test, verbose=0)
pred_classes = np.argmax(preds, axis=1)
true_classes = np.argmax(y_test_categorical, axis=1)
accuracy = np.mean(pred_classes == true_classes)
print(f"Test Accuracy: {accuracy:.4f}")
cm, cm_norm = plot_confusion_matrix(true_classes, pred_classes, class_names, "VGG16 Transfer")
print("\nVGG16 CLASSIFICATION REPORT:")
print("=" * 60)
print(classification_report(true_classes, pred_classes, target_names=class_names, digits=4))
all_results['VGG16 Transfer'] = {
    'pred_classes': pred_classes,
    'true_classes': true_classes,
    'accuracy': accuracy,
    'cm': cm,
    'cm_norm': cm_norm
}

# F1-score hesaplama
print("\n=== SINIF BAZLI F1-SKORLARI (VGG16) ===")
cm_vgg = all_results['VGG16 Transfer']['cm']
f1_scores = []
for i in range(len(class_names)):
    tp = cm_vgg[i, i]; fp = cm_vgg[:, i].sum() - tp; fn = cm_vgg[i, :].sum() - tp
    precision = tp / (tp + fp) if (tp + fp) > 0 else 0
    recall = tp / (tp + fn) if (tp + fn) > 0 else 0
    f1 = 2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else 0
    f1_scores.append(f1)

plt.figure(figsize=(10,5))
plt.bar(class_names, f1_scores, color='teal')
plt.xticks(rotation=45)
plt.ylabel('F1-Score')
plt.title('VGG16 Transfer - Sınıf Bazlı F1-Score', fontweight='bold')
for i, score in enumerate(f1_scores):
    plt.text(i, score + 0.01, f"{score:.2f}", ha='center', fontweight='bold')
plt.ylim(0, 1.05)
plt.grid(True, axis='y', alpha=0.3)
plt.tight_layout(); plt.show()

# Sanity check evaluate()
eval_loss, eval_acc = vgg_transfer_model.evaluate(X_test, y_test_categorical, verbose=0)
print("\n=== TEST ACCURACY SANITY CHECK ===")
print(f"evaluate() acc={eval_acc:.4f} | hesaplanan acc={accuracy:.4f}")

## 10. VGG16 Modeli İçin Grad-CAM Görselleştirme

Transfer learning ile eğittiğimiz VGG16 modelinin test görüntülerinde hangi bölgelere odaklanarak karar verdiğini inceleyelim (Grad-CAM).

In [ ]:
# 10. Adım: VGG16 Transfer Modeli İçin Grad-CAM Görselleştirme


# --- Güvenlik Kontrolü ---
required_symbols = ['vgg_transfer_model', 'X_test', 'y_test_categorical', 'class_names']
for sym in required_symbols:
    if sym not in globals():
        raise RuntimeError(f"Gerekli değişken bulunamadı: {sym}. Lütfen önce ilgili eğitim ve veri hücrelerini çalıştırın.")

# --- 1) Sequential modelden preprocess + base_model (VGG16) + head ayrıştırma ---
preprocess_layer = vgg_transfer_model.get_layer('vgg16_preprocess')
base_model = vgg_transfer_model.get_layer('vgg16')  # Orijinal VGG16 Functional

# Head katmanları (base_model sonrası)
head_layers = []
collect = False
for layer in vgg_transfer_model.layers:
    if layer.name == base_model.name:
        collect = True
        continue
    if collect:
        head_layers.append(layer)

# Hedef konvolüsyon katmanı
try:
    target_conv_layer = base_model.get_layer('block5_conv3')
except Exception as e:
    raise ValueError(f"block5_conv3 katmanı alınamadı: {e}")

# Base modelden hem hedef konv çıkışı hem de base çıkışını dönen alt model
base_feature_model = Model(
    inputs=base_model.input,
    outputs=[target_conv_layer.output, base_model.output],
    name='base_feature_model_for_gradcam'
)

# --- 2) Aktivasyon modeli (preprocess + base + head) ---
cam_input = tf.keras.Input(shape=(IMG_HEIGHT, IMG_WIDTH, IMG_CHANNELS), name='gradcam_input_rgb')
preprocessed = preprocess_layer(cam_input)
conv_acts, base_out = base_feature_model(preprocessed)

x_head = base_out
for hl in head_layers:
    if hl.__class__.__name__ == 'InputLayer':
        continue
    x_head = hl(x_head)
softmax_output = x_head
activation_model = Model(inputs=cam_input, outputs=[conv_acts, softmax_output], name='activation_model_gradcam')
print("Grad-CAM için activation model hazır:")
print(" - Conv aktivasyon shape:", activation_model.output[0].shape)
print(" - Softmax shape:", activation_model.output[1].shape)

# --- 3) Grad-CAM Hesaplayıcı ---

def generate_grad_cam(img_array, class_index=None, eps=1e-8):
    """
    img_array: (H,W,3) veya (1,H,W,3) RGB (0-255 range)
    class_index: None -> modelin tahmin ettiği sınıf
    Dönüş: heatmap(0-1), pred_vector
    """
    if img_array.ndim == 3:
        input_tensor = np.expand_dims(img_array, axis=0).astype('float32')
    else:
        input_tensor = img_array.astype('float32')

    input_tf = tf.convert_to_tensor(input_tensor)
    with tf.GradientTape() as tape:
        conv_out, preds = activation_model(input_tf, training=False)
        if class_index is None:
            class_index = tf.argmax(preds[0])
        class_channel = preds[:, class_index]
    grads = tape.gradient(class_channel, conv_out)  # (1,h,w,c)

    pooled_grads = tf.reduce_mean(grads, axis=(0,1,2))  # (c,)
    conv_out = conv_out[0]  # (h,w,c)

    heatmap = tf.reduce_sum(conv_out * pooled_grads, axis=-1)
    heatmap = tf.nn.relu(heatmap).numpy()

    if np.all(heatmap == 0):
        heatmap = heatmap + eps

    heatmap -= heatmap.min()
    max_val = heatmap.max()
    if max_val > 0:
        heatmap /= max_val

    heatmap = cv2.resize(heatmap, (img_array.shape[1], img_array.shape[0]))
    return heatmap, preds.numpy()[0]

# --- 4) Isı haritasını bindirme ---

def overlay_heatmap(img, heatmap, alpha=0.45, colormap=cv2.COLORMAP_JET):
    base = img.astype('float32')
    if base.max() > 1.0:
        base = base / 255.0
    heat_uint8 = np.uint8(255 * heatmap)
    color = cv2.applyColorMap(heat_uint8, colormap)
    color = cv2.cvtColor(color, cv2.COLOR_BGR2RGB).astype('float32') / 255.0
    overlay = alpha * color + (1 - alpha) * base
    overlay = np.clip(overlay, 0, 1)
    return overlay

# --- 5) Her sınıftan bir test örneği seç ---
y_true_test = np.argmax(y_test_categorical, axis=1)
indices_per_class = {}
for idx, cls in enumerate(y_true_test):
    if cls not in indices_per_class:
        indices_per_class[cls] = idx
    if len(indices_per_class) == len(class_names):
        break
selected_indices = list(indices_per_class.values())
print(f"Her sınıftan seçilen test indeksleri: {selected_indices}")

# --- 6) Parametreler ---
USE_TRUE_CLASS_FOR_GRADCAM = False  # True -> Grad-CAM gerçek sınıf için hesaplanır

# --- 7) Görselleştirme ---
num_samples = len(selected_indices)
plt.figure(figsize=(4 * num_samples, 5))
for i, idx in enumerate(selected_indices):
    img = X_test[idx]
    true_cls = y_true_test[idx]

    heat_grad, pred_vec = generate_grad_cam(img, class_index=true_cls if USE_TRUE_CLASS_FOR_GRADCAM else None)
    pred_class = np.argmax(pred_vec)
    pred_prob = pred_vec[pred_class]
    grad_overlay = overlay_heatmap(img, heat_grad)

    # Orijinal
    plt.subplot(2, num_samples, i + 1)
    title_lines = [f"Gerçek: {class_names[true_cls]}", f"Tahmin: {class_names[pred_class]} ({pred_prob:.2f})"]
    if USE_TRUE_CLASS_FOR_GRADCAM and pred_class != true_cls:
        title_lines.append("(GC hedef=gerçek)")
    plt.imshow(img.astype('uint8'))
    plt.title("\n".join(title_lines), fontsize=10, fontweight='bold')
    plt.axis('off')

    # Grad-CAM Overlay
    plt.subplot(2, num_samples, num_samples + i + 1)
    plt.imshow(grad_overlay)
    plt.title('Grad-CAM', fontsize=10, fontweight='bold')
    plt.axis('off')

plt.suptitle('VGG16 Transfer Modeli - Grad-CAM Odak Alanları', fontweight='bold', fontsize=16)
plt.tight_layout(rect=[0, 0, 1, 0.92])
plt.show()

print('Grad-CAM görselleştirme tamamlandı.')

## 11. Prediction Verisi Üzerinde Test (VGG16)

`seg_pred` klasöründeki bağımsız prediction görüntüleri üzerinde VGG16 modelinin tahminlerini inceleyelim.

In [ ]:
# Prediction verisi üzerinde VGG16 tahminleri


prediction_path = pred_path  # Önceden tanımlandı: /kaggle/input/intel-image-classification/seg_pred/seg_pred
print(f"Prediction klasörü: {prediction_path}")

# Prediction klasöründeki görüntü dosyalarını listele
all_pred_files = [f for f in os.listdir(prediction_path) if f.lower().endswith(('.jpg', '.jpeg', '.png'))]
print(f"Toplam prediction görüntüsü: {len(all_pred_files)}")

# Çok fazla görsel varsa görselleştirme için ilk N tanesini seçebilirsiniz
VISUALIZE_COUNT = 18  # 3x6 grid
selected_files = all_pred_files[:VISUALIZE_COUNT]

results = []

def infer_possible_true_label(filename, class_names):
    name_lower = filename.lower()
    for cls in class_names:
        if cls in name_lower:
            return cls
    return None

images = []
for fname in selected_files:
    img_path = os.path.join(prediction_path, fname)
    img = cv2.imread(img_path)
    if img is None:
        continue
    img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    img_rgb = cv2.resize(img_rgb, (IMG_WIDTH, IMG_HEIGHT))
    images.append((fname, img_rgb))

# Batch halinde tahmin yapmak için array'e çevir (0-255 float -> Lambda preprocess işleyecek)
if images:
    batch_array = np.stack([img for (_, img) in images]).astype('float32')
    preds = vgg_transfer_model.predict(batch_array, verbose=0)
else:
    preds = []

for i, (fname, img_rgb) in enumerate(images):
    pred_vector = preds[i]
    predicted_class = class_names[np.argmax(pred_vector)]
    predicted_prob = np.max(pred_vector)
    possible_true = infer_possible_true_label(fname, class_names)

    results.append({
        'filename': fname,
        'predicted_class': predicted_class,
        'predicted_prob': predicted_prob,
        'possible_true_from_name': possible_true
    })

# Görselleştirme
cols = 6
rows = int(np.ceil(len(images) / cols))
plt.figure(figsize=(3*cols, 3*rows))
for idx, (fname, img_rgb) in enumerate(images):
    pred_class = results[idx]['predicted_class']
    pred_prob = results[idx]['predicted_prob']
    possible_true = results[idx]['possible_true_from_name']
    title = f"Tahmin: {pred_class}\nP={pred_prob:.2f}"
    if possible_true is not None and possible_true != pred_class:
        title += f"\n(Muht. Gerçek: {possible_true})"
    elif possible_true is not None and possible_true == pred_class:
        title += f"\n(İsim Eşleşti)"

    plt.subplot(rows, cols, idx + 1)
    plt.imshow(img_rgb.astype('uint8'))
    plt.title(title, fontsize=9, fontweight='bold')
    plt.axis('off')
plt.suptitle('VGG16 Transfer Modeli - Prediction Görselleri Tahminleri', fontweight='bold', fontsize=14)
plt.tight_layout(rect=[0,0,1,0.95])
plt.show()

# Sonuçları DataFrame olarak özetle
results_df = pd.DataFrame(results)

print("Prediction değerlendirmesi tamamlandı.")
